In [2]:
import pandas as pd
import os
import csv
import numpy as np

In [16]:
def average(data):
    df=pd.DataFrame()
    data_columns= data.columns #cabeceras
    comparador = int(data.iloc[0][0]) #nos quedamos con la parte entera del timestamp CPU

    cont = -1
    n=0
    sumas = np.zeros(shape=22)
    
    for column in data_columns:
        df[column]= [] #metemos las columnas en el DataFrame
    sit=0
    
    while (cont<len(data.axes[0])):
        if(cont>-1):
            row= data.iloc[cont]
            timestamp = int(row[0])
            if((timestamp == comparador) and (cont<(len(data.axes[0])-1))):
                col=0
                for column in row:
                    sumas[col]= sumas[col] + column
                    col= col+1
                n=n+1
            else:
                col = 0
                for num in sumas: #cada pos de sumas
                    sumas[col]= num/n #media de las columnas con mismo timestamp
                    col= col +1 
    
                df.loc[sit] = sumas
                sit = sit + 1      

                col=0

                sumas = np.zeros(shape=22)
                comparador = int(row[0]) #comparar con el siguiente
                for column in row:
                    sumas[col]= column
                    col= col+1
                n = n+1
                
        cont=cont+1
    return df

In [4]:
def find_delimiter(filename):
    sniffer= csv.Sniffer()
    with open(filename) as fp:
        delimiter = sniffer.sniff(fp.read(5000)).delimiter
    return delimiter

In [13]:
def clean_data(input_file, output_file):
    #Apertura del archivo con Pandas DataFrame 
    #Detección del separador de datos
    separador= find_delimiter(input_file)
    df=pd.read_csv(input_file, sep= separador)

    headers = df.columns
    filac = df.iloc[0]

    #Detección de si la segunda fila son datos o cabeceras
    isFloat= isinstance(filac.iloc[0], float)
    
    nuevas = [] 
    contador = 0
    if (isFloat==False):
        #Combinación de las cabeceras de datos con sus correspondientes unidades, eliminando la segunda cabecera 
        for cabeceras in headers: 
            cabeceras = cabeceras + " (" + filac.iloc[contador] + ")"
            nuevas.append(cabeceras)
            contador = contador +1
        df.columns= nuevas
        df = df.drop([0])
    

    #Eliminación de filas con NaN
    clean_df = average(df.dropna(axis=0))
    
    #Escribe un los datos filtrados en un archivo nuevo 
    clean_df.to_csv(output_file, index=False, sep = separador)
    print("Datos limpios guardados como ", output_file)

In [30]:
def main(): 
    while True: 
        # Solicitar al usuario que ingrese el nombre del archivo de entrada
        input_file = input("Ingrese la ruta y nombre del archivo CSV de entrada: ")
    
         # Solicitar al usuario que ingrese el nombre del archivo de salida
        output_file = input("Ingrese la ruta y el nombre del archivo CSV de salida para los datos limpios: ")
    
        if input_file == output_file: 
            print("Por favor, indique nombres para los archivos que sean distintivos")
        else: 
            # Llamar a la función para limpiar los datos
            clean_data(input_file, output_file)
            break

In [31]:
if __name__ == "__main__":
    main()

Ingrese la ruta y nombre del archivo CSV de entrada:  C:\Users\isagl\anaconda3\envs\tfg_shimmer\datosCSVleer/default_exp_Session1_Shimmer_F693_Calibrated_SD.csv
Ingrese la ruta y el nombre del archivo CSV de salida para los datos limpios:  C:\Users\isagl\anaconda3\envs\tfg_shimmer\datosCSVleer/limpios.csv


Datos limpios guardados como  C:\Users\isagl\anaconda3\envs\tfg_shimmer\datosCSVleer/limpios.csv
